In [1]:
# 사용할 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 전 기본 세팅
#한글 깨짐
plt.rc("font", family="Malgun Gothic")

In [2]:
# 파일 불러오기
sample_submission = pd.read_csv('data/sample_submission.csv', index_col='no')
test = pd.read_csv('data/test.csv', index_col='no')
train = pd.read_csv('data/train.csv', index_col='no')

## 예측할 값 : 개인 소득
- income : 50K 초과는 1, 50K 이하는 0 (소득)

### 컬럼 의미
- age - continuous.
- workclass(일 유형) : Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt(일련의 관측 결과를 바탕으로 인구조사국이 부여하는 개인의 가중치): continuous.
- education(교육수준) : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num(교육수준 번호) : continuous.
- marital-status(결혼 상태) : Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation(직업) : Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship(가족관계) : Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race(인종) : White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex(성별) : Female, Male.
- capital-gain(자본 이익) : continuous.
- capital-loss(자본 손실) : continuous.
- hours-per-week(주당 근무 시간) : continuous.
- native-country(국적) : United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

### value값 띄어쓰기 제거

In [3]:
# 띄어쓰기 제거함
for i in train.columns : 
    if train[f'{i}'].dtype == 'object' :
        train[f'{i}'] = train[f'{i}'].apply(lambda x: x.replace(' ', '')) # 제거하는 함수
    else :
        train[f'{i}'] = train[f'{i}']
        
for i in test.columns : 
    if test[f'{i}'].dtype == 'object' :
        test[f'{i}'] = test[f'{i}'].apply(lambda x: x.replace(' ', '')) # 제거하는 함수
    else :
        test[f'{i}'] = test[f'{i}']

In [4]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', '?',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
cover_title=['Private', 'Self-emp-not-inc', 'Local-gov', 'Never-worked',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
title_dict = dict(zip(title, cover_title))
title_dict
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] =test['workclass'].map(title_dict)

In [5]:
title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       '?', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
cover_title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'United-States', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
title_dict = dict(zip(title, cover_title))
title_dict
train['native-country'] = train['native-country'].map(title_dict)

test['native-country'] = test['native-country'].map(title_dict)

In [6]:
title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',
       'Armed-Forces']
cover_title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', 'Prof-specialty', 'Craft-repair', 'Sales',
       'Armed-Forces']
title_dict = dict(zip(title, cover_title))
title_dict
train['occupation'] = train['occupation'].map(title_dict)
test['occupation'] = test['occupation'].map(title_dict)

In [7]:
train = train.drop('education',axis=1)
test = test.drop('education',axis=1)

In [8]:
marital_status = ['Divorced', 'Never-married', 'Married-civ-spouse', 'Separated',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse']
conver_marital_status = ['not-married', 'not-married', 'married', 'not-married', 'not-married', 'not-married','married']
marital_status_dict=dict(zip(marital_status, conver_marital_status))

train['marital-status']=train['marital-status'].map(marital_status_dict)
test['marital-status']=test['marital-status'].map(marital_status_dict)

In [9]:
train['relationship'].unique()
title=['Not-in-family', 'Husband', 'Unmarried', 'Other-relative',
       'Own-child', 'Wife']
cover_title=['Not-in-family', 'Married', 'Unmarried', 'Other-relative',
       'Own-child', 'Married']
title_dict = dict(zip(title, cover_title))
train['relationship'] = train['relationship'].map(title_dict)

title=['Own-child', 'Wife', 'Husband', 'Not-in-family', 'Unmarried',
       'Other-relative']
cover_title=['Own-child', 'Married', 'Married', 'Not-in-family', 'Unmarried',
       'Other-relative']
title_dict = dict(zip(title, cover_title))
test['relationship'] = test['relationship'].map(title_dict)

In [10]:
bins = [0,4,8,12,16] # 범위지정
labels = ['e1', 'e2', 'e3', 'e4'] # 라벨지정
train['education-num'] = pd.cut(train['education-num'], bins=bins, labels = labels)
test['education-num'] = pd.cut(test['education-num'], bins=bins, labels=labels)

In [11]:
"""#  사회 초년생 - age1 [17~30]
age1 = [i for i in range(17, 31)]
cover_age1 = ['young'] * (31-17)

#  중년층 - age2 [31~50]
age2 = [i for i in range(31, 51)]
cover_age2 = ['middle'] * (51-31)

# 노년층 - age3 [51 ~ 90]
age3 = [i for i in range(51, 91)]
cover_age3 = ['old'] * (91-51)

age_dict = dict(zip(age1 + age2 + age3, cover_age1 + cover_age2 + cover_age3))

train['age']=train['age'].map(age_dict)
test['age']=test['age'].map(age_dict)"""

"#  사회 초년생 - age1 [17~30]\nage1 = [i for i in range(17, 31)]\ncover_age1 = ['young'] * (31-17)\n\n#  중년층 - age2 [31~50]\nage2 = [i for i in range(31, 51)]\ncover_age2 = ['middle'] * (51-31)\n\n# 노년층 - age3 [51 ~ 90]\nage3 = [i for i in range(51, 91)]\ncover_age3 = ['old'] * (91-51)\n\nage_dict = dict(zip(age1 + age2 + age3, cover_age1 + cover_age2 + cover_age3))\n\ntrain['age']=train['age'].map(age_dict)\ntest['age']=test['age'].map(age_dict)"

In [12]:
bins = [i for i in range(0, 101,10)]
labels = ['age' + str(i) for i in range(0, 100,10)] # 라벨지정
train['age'] = pd.cut(train['age'], bins=bins, labels = labels)
test['age'] = pd.cut(test['age'], bins=bins, labels=labels)

In [13]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', 'State-gov',
       'Self-emp-inc', 'Federal-gov', 'Without-pay', 'Never-worked']
cover_title=['Private', 'Self-emp', 'gov', 'gov',
       'Self-emp', 'gov', 'beggar', 'beggar']
title_dict = dict(zip(title, cover_title))
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] = test['workclass'].map(title_dict)

In [14]:
native_country = ['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Columbia', 'Hungary', 'Portugal', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
conver_native_country = ['United-States', 'North America', 'Asia', 'Europe', 'North America',
                         'North America', 'North America', 'North America', 'Asia', 'Europe',
                         'North America', 'Asia', 'Asia', 'North America', 'Europe',
                         'Asia', 'South America', 'North America', 'Asia', 'Europe',
                        'North America', 'Europe', 'Asia', 'Asia', 'North America',
                         'South America', 'Europe', 'Europe', 'South America', 'Asia',
                         'Europe', 'Asia', 'North America', 'Asia', 'Europe',
                         'Asia', 'South America', 'Europe', 'North America', 'Europe', 'Europe']

native_country_dict = dict(zip(native_country, conver_native_country))
native_country_dict
train['native-country']=train['native-country'].map(native_country_dict)
test['native-country']=test['native-country'].map(native_country_dict)

In [15]:
train = train.drop('fnlwgt',axis=1)
test= test.drop('fnlwgt',axis=1)

- 컬럼순서 바꾸기

In [16]:
# 문제/답 분리
X_train = train.loc[:,:'native-country']
y_train = train['income']

In [17]:
X_train = pd.get_dummies(X_train) # 0과 1로만 이루어진 열을 생성
X_test = pd.get_dummies(test)

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

그리드 서치

In [19]:
Ada_model = AdaBoostClassifier()
Ada_param_grid = {    
    'n_estimators' : [50,150,250],
    'learning_rate' : [0.5,1]
}
Ada_grid = GridSearchCV(Ada_model,
                        Ada_param_grid,
                        cv = 3)
Ada_grid.fit(X_train,y_train)
# best_score_ 메서드
print(Ada_grid.best_score_)
# best_params_ 메서드
print(Ada_grid.best_params_)

0.8631290781843551
{'learning_rate': 1, 'n_estimators': 250}


In [21]:
lgbm = LGBMClassifier()

params = {
    'boosting_type': ['gbdt', 'dart'], # defalt 'gbdt'
    'num_leaves': [20, 31, 50, 70], # default 31
    'max_depth': [-1, 5, 7, 10, 15, 20, 30], # default -1. 끝까지 만드는 것
    'learning_rate': [0.001, 0.01, 0.05, 0.1], # defaut 0.1
}

lgbm_grid_cv = GridSearchCV(lgbm, param_grid=params, scoring='f1', n_jobs=-1, cv=5, verbose=1)
lgbm_grid_cv.fit(X_train,y_train)
print('GridSearchCV 최고 평균 정확도 수치: {:.4f}'.format(lgbm_grid_cv.best_score_))
print('GridSearchCv 최적 하이퍼 파라미터: ', lgbm_grid_cv.best_params_)

Fitting 5 folds for each of 224 candidates, totalling 1120 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000863 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 353
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
GridSearchCV 최고 평균 정확도 수치: 0.6999
GridSe

In [27]:
# 그리드서치 바탕으로 하이퍼파라미터 조정
final_Ada_model = AdaBoostClassifier(n_estimators=250,
                              learning_rate=1)
final_Ada_model.fit(X_train,y_train)
pre = final_Ada_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('final_ada_model.csv', index = False)

In [25]:
final_lgbm_model = lgbm_grid_cv.best_estimator_
final_lgbm_model.fit(X_train, y_train)
pre = final_lgbm_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('final_lgbm_model.csv', index = False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000900 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 353
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num

In [32]:
final_cb_model = CatBoostClassifier(verbose=0)
final_cb_model.fit(X_train, y_train)
pre = final_cb_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('final_cb_model.csv', index = False)

In [33]:
final_voting_model = VotingClassifier(
                        estimators=[
                        ('final_Ada_mode',final_Ada_model),
                        ('final_lgbm_model',final_lgbm_model),
                        ('final_cb_model',final_cb_model)
                        ],
                        voting='soft'
                        )
# 학습
final_voting_model.fit(X_train,y_train)
# 예측
pre = final_voting_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('final_voting_model.csv', index = False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001025 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 353
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num